THA22530230 (SUDIP THAPA MAGAR)

20/02/2024

Lets Begain..!!!

In [47]:
from collections import deque
from queue import PriorityQueue
import heapq
from datetime import datetime
from tabulate import tabulate

In [48]:
#Task_1:Defining the graph for the transportation network.

graph = {
    'Manchester': {'Liverpool': 40, 'York': 60, 'Carlisle': 120, 'Edinburgh': 220, 'Newcastle': 130},
    'Liverpool': {'Manchester': 40, 'Holyhead': 90},
    'Holyhead': {'Liverpool': 90},
    'York': {'Manchester': 60, 'Newcastle': 80},
    'Carlisle': {'Manchester': 120, 'Glasgow': 100},
    'Newcastle': {'York': 80, 'Edinburgh': 110, 'Manchester': 130},
    'Glasgow': {'Carlisle': 100, 'Oban': 90, 'Edinburgh': 40, 'Aberdeen': 140, 'Inverness': 170},
    'Edinburgh': {'Newcastle': 110, 'Glasgow': 40, 'Manchester': 220},
    'Oban': {'Glasgow': 90, 'Inverness': 110},
    'Aberdeen': {'Glasgow': 140, 'Inverness': 110},
    'Inverness': {'Oban': 110, 'Aberdeen': 110, 'Glasgow': 170}
    
}

# Collecting user input for starting and destination cities.
def Get_User_Input(graph):
    print(" Welcome to AI Car Service. We currenlty only serving 11 cities as available in the list below.")
    print("Service Covered cities:", ', '.join(graph.keys()))
    
    while True:
        starting_city = input("Travel from: ").capitalize()
        if starting_city not in graph:
            print(f"{starting_city} is a invalid city name. Please choose from the list.")
            continue
        break
    
    while True:
        ending_city = input("Travel to: ").capitalize()
        if ending_city not in graph:
            print(f"{ending_city} is a invalid city name. Please choose from the list.")
            continue
        elif ending_city == starting_city:
            print("Destination city cannot be the same as Departure city. Please choose a different city.")
            continue
        break
    
    return starting_city, ending_city

In [50]:
#Task_2: Implementing_Search_Algorithms

#Depth-First Search (DFS)

def Find_Shortest_Path(graph, current_vertex, target_vertex, visited_vertices=[], total_distance=0):
    visited_vertices = visited_vertices + [current_vertex]
    if current_vertex == target_vertex:
        return visited_vertices, total_distance
    if current_vertex not in graph:
        return None, 0
    shortest_path = None
    shortest_path_distance = float('inf')
    for neighbor in graph[current_vertex]:
        if neighbor not in visited_vertices:
            new_path, new_distance = Find_Shortest_Path(graph, neighbor, target_vertex, visited_vertices, total_distance + graph[current_vertex][neighbor])
            if new_path:
                if not shortest_path or new_distance < shortest_path_distance:
                    shortest_path = new_path
                    shortest_path_distance = new_distance
    return shortest_path, shortest_path_distance


#Breadth-First Search (BFS)

def Find_Shortest_Path_Bfs(graph, start_vertex, target_vertex):
    queue = deque([(start_vertex, [start_vertex], 0)]) 
    while queue:
        current_vertex, path, total_distance = queue.popleft() 
        if current_vertex == target_vertex:
            return path, total_distance
        for adjacent_vertex, distance in graph[current_vertex].items():
            if adjacent_vertex not in path: 
                new_path = path + [adjacent_vertex]
                new_distance = total_distance + distance
                queue.append((adjacent_vertex, new_path, new_distance))  # Add new state to queue
    return None, 0  # Return None, 0 if no path is found



#A* Search

def Calculate_Heuristic(graph, current_vertex, target_vertex):
    # Check if a direct edge exists from the current vertex to the target vertex.
    if target_vertex in graph[current_vertex]:
        return graph[current_vertex][target_vertex]

    minimal_edge_distance = min(graph[current_vertex].values())

    return minimal_edge_distance

def Find_Path_A_Star(graph, start_vertex, target_vertex):
    priority_queue = PriorityQueue()
    priority_queue.put((0, start_vertex, [start_vertex], 0))
    visited = set()

    while not priority_queue.empty():
        _, current_vertex, path, total_distance = priority_queue.get()

        if current_vertex in visited:
            continue
        if current_vertex == target_vertex:
            return path, total_distance

        visited.add(current_vertex)

        for adjacent_vertex, distance in graph[current_vertex].items():
            if adjacent_vertex not in visited:
                new_distance = total_distance + distance
                estimated_remaining_distance = Calculate_Heuristic(graph, adjacent_vertex, target_vertex)
                priority_queue.put((new_distance + estimated_remaining_distance, adjacent_vertex, path + [adjacent_vertex], new_distance))

    return None, 0  # Return None, 0 if no path is found



#Dijkstra Algorithm

def Find_Shortest_Path_Dijkstra(graph, start_vertex, target_vertex):
    priority_queue = [(0, start_vertex, [start_vertex])]
    visited_vertices = set()  # To keep track of visited vertices to avoid revisiting them
    while priority_queue:
        current_distance, current_vertex, current_path = heapq.heappop(priority_queue)
        if current_vertex == target_vertex:
            return current_path, current_distance
        if current_vertex in visited_vertices:
            continue
        visited_vertices.add(current_vertex)
        for adjacent_vertex, edge_weight in graph[current_vertex].items():
            if adjacent_vertex not in visited_vertices:
                new_distance = current_distance + edge_weight
                new_path = current_path + [adjacent_vertex]
                heapq.heappush(priority_queue, (new_distance, adjacent_vertex, new_path))
    return None, 0




In [51]:
#Task 3. Results
# Just Assuming that:
emissions_factor = 0.21  # Average kg CO2 emitted per mile by a passenger vehicle
urban_aqi = 70  # An arbitrary Air Quality Index value assumed for urban areas
rural_aqi = 30  # An arbitrary Air Quality Index value assumed for rural areas
green_space_percentage = {  # Assumed percentage of green spaces for each city
    'Manchester': 23, 'Liverpool': 22, 'Holyhead': 45, 'York': 38,
    'Carlisle': 40, 'Newcastle': 29, 'Glasgow': 31, 'Edinburgh': 23,
    'Oban': 59, 'Aberdeen': 41, 'Inverness': 52
}

def Accumulate_Search_Results(results, algorithm_name, path, total_distance):
    if path:
        cost_per_mile = 2  # £2 per mile
        total_cost = total_distance * cost_per_mile
        
        # Estimate carbon emissions
        total_emissions = total_distance * emissions_factor
        
        # Average AQI along the route
        avg_aqi = sum(urban_aqi if city in ['Manchester', 'Liverpool', 'Newcastle', 'Glasgow', 'Edinburgh'] else rural_aqi for city in path) / len(path)
        
        # Estimate green space percentage along the route
        avg_green_space = sum(green_space_percentage[city] for city in path) / len(path)
        
        results.append([algorithm_name, " -> ".join(path), total_distance, f"£{total_cost}", f"{total_emissions:.2f} kg", avg_aqi, f"{avg_green_space:.0f}%"])
    else:
        results.append([algorithm_name, "No path found", 0, "£0", "0 kg", "N/A", "N/A"])
def main():
    start_city, end_city = Get_User_Input(graph)  # This function will be called to get user input
    current_date = datetime.now().strftime("%Y-%m-%d")
    print(f"You are traveling from {start_city} to {end_city} on {current_date}.")

    # Initialized an empty list to accumulate results
    search_results = []

    # Accumulating results from each algorithms
    path_dfs, distance_dfs = Find_Shortest_Path(graph, start_city, end_city)
    Accumulate_Search_Results(search_results, "DFS", path_dfs, distance_dfs)

    path_bfs, distance_bfs = Find_Shortest_Path_Bfs(graph, start_city, end_city)
    Accumulate_Search_Results(search_results, "BFS", path_bfs, distance_bfs)

    path_a_star, distance_a_star = Find_Path_A_Star(graph, start_city, end_city)
    Accumulate_Search_Results(search_results, "A*", path_a_star, distance_a_star)

    path_dijkstra, distance_dijkstra = Find_Shortest_Path_Dijkstra(graph, start_city, end_city)
    Accumulate_Search_Results(search_results, "Dijkstra", path_dijkstra, distance_dijkstra)

    # Printing Result in a table.
    print(tabulate(search_results, headers=["Algorithm", "Path", "Total Distance (miles)", "Total Cost", "Carbon Emissions", "Avg. AQI", "Green Space %"], tablefmt="pretty"))
if __name__ == "__main__":
    main()


 Welcome to AI Car Service. We currenlty only serving 11 cities as available in the list below.
Service Covered cities: Manchester, Liverpool, Holyhead, York, Carlisle, Newcastle, Glasgow, Edinburgh, Oban, Aberdeen, Inverness
You are traveling from Manchester to Oban on 2024-03-23.
+-----------+-------------------------------------------+------------------------+------------+------------------+----------+---------------+
| Algorithm |                   Path                    | Total Distance (miles) | Total Cost | Carbon Emissions | Avg. AQI | Green Space % |
+-----------+-------------------------------------------+------------------------+------------+------------------+----------+---------------+
|    DFS    | Manchester -> Carlisle -> Glasgow -> Oban |          310           |    £620    |     65.10 kg     |   50.0   |      38%      |
|    BFS    | Manchester -> Carlisle -> Glasgow -> Oban |          310           |    £620    |     65.10 kg     |   50.0   |      38%      |
|    A*